In [ ]:
!pip install azure-storage-blob --upgrade
!pip install hana_ml
!pip install replace

In [ ]:
import time
import replace
from azure.storage.blob import BlobServiceClient
import pandas as pd
import hana_ml.dataframe as dataframe
STORAGEACCOUNTURL= 'https://XXXXXX.blob.core.windows.net'  #Replace XXXXXX with your Azure Storage Account
STORAGEACCOUNTKEY= '<Insert your Storage Account Access Key>' # Insert your access keys for your Storage Account
LOCALFILENAME= 'Jancsv'  #This could be anyname . Just for local download
CONTAINERNAME= 'xxxx'   #Insert your container name 
t1=time.time()
blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
#Provide you connection details below either to ingest data into DWC Container or to HANA Cloud 
conn = dataframe.ConnectionContext(address = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX', #Provide your OpenSQL Host here
                                   port = 443, 
                                   user = 'XXXXXX', #Provide your OpenSQL container Username here 
                                   password = 'XXXXXXX', #Provide your OpenSQL contianer password here
                                   encrypt = 'true'
                                  )
container = blob_service_client_instance.get_container_client(CONTAINERNAME)
blob_list = container.list_blobs()
print(blob_list)
for blob in blob_list:
  blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, blob.name, snapshot=None)
  with open(blob.name, "wb") as my_blob:
      blob_data = blob_client_instance.download_blob()
      blob_data.readinto(my_blob)
      t2=time.time()
print(("It takes %s seconds to download "+blob.name) % (t2 - t1))

  # LOCALFILE is the file path

  dataframe_blobdata = pd.read_csv(blob.name)

  key = blob.name
  keystr = key.replace(".csv","")

  print(dataframe_blobdata)
#Insert into DWC Open SQL container 
  df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = dataframe_blobdata, 
                                                   table_name = keystr,
                                                   #schema = 'XXXX',   #Schema name is not needed for DWC Open SQL Container 
                                                   force = True,
                                                   replace = True)

